In [87]:
import pandas as pd
import numpy as np
from math import ceil

In [88]:
teams = pd.read_csv("2021_teams.csv")
games = pd.read_csv("./games.csv")

In [89]:
numTeams = len(teams)
numGames = len(games)

In [90]:
# Create the Colley matrix

colley_matrix = 2*np.diag(np.ones(numTeams))
b = np.ones(numTeams)
k = 0


In [91]:
day_before_season = games.loc[0,'days_since_timestart'] - 1
last_day_of_season = games.loc[len(games)-1,'days_since_timestart']

In [92]:
weight_home_win = 1
weight_away_win = 1
weight_neutral_win = 1
segment_weights = [1,1,2,3,5]
num_segments = len(segment_weights)

In [93]:
for i in range(numGames):
  [current_day,date,team_1_id, team_1_homefield, team_1_score, team_2_id, team_2_homefield, team_2_score] = games.loc[i,:].values

  team_1_index = team_1_id - 1
  team_2_index = team_2_id - 1

  weight_index = ceil(num_segments*((current_day-day_before_season)/(last_day_of_season-day_before_season)))-1
  time_weight = segment_weights[weight_index]

  # If Team 1 won the game
  if team_1_score > team_2_score:
    # If team 1 was the home team
    if team_1_homefield == 1:
      gameWeight = weight_home_win * time_weight
    # If team 1 was the away team
    elif team_1_homefield == -1:
      gameWeight = weight_away_win * time_weight
    # If team 1 was a neutral team
    else:
      gameWeight = weight_neutral_win * time_weight
  else:
    if team_2_homefield == 1:
      gameWeight = weight_home_win * time_weight
    elif team_2_homefield == -1:
      gameWeight = weight_away_win * time_weight
    else:
      gameWeight = weight_neutral_win * time_weight

  # Update the Colley Matrix and RHS
  colley_matrix[team_1_index,team_2_index] -= gameWeight
  colley_matrix[team_2_index,team_1_index] -= gameWeight

  colley_matrix[team_1_index,team_1_index] += gameWeight
  colley_matrix[team_2_index,team_2_index] += gameWeight

  if team_1_score > team_2_score:
    b[team_1_index] += gameWeight/2
    b[team_2_index] -= gameWeight/2
  else:
    b[team_1_index] -= gameWeight/2
    b[team_2_index] += gameWeight/2

In [94]:
# Calculate the linear system
r = np.linalg.solve(colley_matrix,b)
iSort = np.argsort(-r)

In [95]:
print('\n\n************** COLLEY Rating Method **************\n')
print('===========================')
print('Rank   Rating    Team   ')
print('===========================')
if k==0:
    numberTeamToPrint = numTeams
else:
    numberTeamToPrint = k

colley_ratings = pd.DataFrame(columns=['rank','rating','team'])

for i in range(numberTeamToPrint):
    print(f'{i+1:4d}   {r[iSort[i]]:.5f}  {teams.loc[iSort[i],"team"]}')
    colley_ratings.loc[i,:] = [i+1,r[iSort[i]],teams.loc[iSort[i],"team"]]

print('')   # extra carriage return



************** COLLEY Rating Method **************

Rank   Rating    Team   
   1   1.06427  Arizona
   2   1.04650  Villanova
   3   1.04313  Tennessee
   4   1.02959  Gonzaga
   5   1.01962  Kansas
   6   1.00469  Boise_St
   7   0.98926  Kentucky
   8   0.98441  Houston
   9   0.97640  Purdue
  10   0.97392  Arkansas
  11   0.97115  UCLA
  12   0.96410  Baylor
  13   0.96142  St_Mary's_CA
  14   0.95839  Providence
  15   0.95721  Auburn
  16   0.94892  Colorado_St
  17   0.94692  Duke
  18   0.93060  San_Diego_St
  19   0.92321  Memphis
  20   0.91947  Wisconsin
  21   0.91224  Connecticut
  22   0.91185  Iowa
  23   0.90932  Texas_Tech
  24   0.88366  Illinois
  25   0.88362  Davidson
  26   0.87772  USC
  27   0.87677  Creighton
  28   0.87569  North_Carolina
  29   0.85417  Murray_St
  30   0.85183  VCU
  31   0.84586  Virginia_Tech
  32   0.84548  Texas_A&M
  33   0.84007  SMU
  34   0.83244  Seton_Hall
  35   0.83220  Wyoming
  36   0.83092  S_Dakota_St
  37   0.81852  UAB
 

In [96]:
colley_ratings.to_csv("colley_ratings.csv",index=False)